In [3]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  1595k      0 --:--:-- --:--:-- --:--:-- 1597k


In [10]:
%pip install tiktoken torch numpy

     --------------------------------------- 14.8/14.8 MB 18.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\herlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [18]:
# Build vocabulary
# import tiktoken

# enc = tiktoken.get_encoding('gpt2')
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("vocab_size", vocab_size)

vocab_size 65


In [19]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a strin

# encode = enc.encode
# decode = enc.decode

print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [20]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [21]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [22]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when the input is {context.tolist()}  the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
when the input is [24]  the target: 43
when the input is [24, 43]  the target: 58
when the input is [24, 43, 58]  the target: 5
when the input is [24, 43, 58, 5]  the target: 57
when the input is [24, 43, 58, 5, 57]  the target: 1
when the input is [24, 43, 58, 5, 57, 1]  the target: 46
when the input is [24, 43, 58, 5, 57, 1, 46]  the target: 43
when the input is [24, 43, 58, 5, 57, 1, 46, 43]  the target: 39
when the input is [44]  the target: 53
when the input is [44, 53]  the target: 56
when the input is [44, 53, 56]  the target: 1
when the input is [44, 53, 56, 1]  the target: 58
when the in

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss
    
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)

      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)

# Generate text
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

NameError: name 'vocab_size' is not defined

In [32]:
optimizer = torch.optim.AdamW(m.parameters(),  lr=1e-3)

In [46]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


print(loss.item())

2.4538254737854004


In [49]:
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


PUE:
Fistentite'shanepano quno norcou,
Thitroce t, wn'lorpis Ask; fonthars thintodet buer?
And betondape t h IOUSTEY it n umerll burane, gothathil tw

Angl fforn'V:
Clathores?
METhas tinse habusey me falomar bowonond'd.
Cis dard pr w thisereme!
The't; l, he y och 'eint my tin;
OLONG bllour bu s pesoworolll to spursin:

NGo icht ilougifay my? be uere af t ado be noucishan amen'thro ais me.
I d sung


In [1]:
import torch
torch.manual_seed(1337)
B,T,C = 4, 8,2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [2]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [3]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x 

In [14]:
import torch
from torch.nn import functional as F

torch.manual_seed(1337)
B, T, C = 4, 8, 42
x = torch.rand(B, T, C)

# Single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k =  key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v


out.shape

torch.Size([4, 8, 16])

In [15]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [16]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4956, 0.5044, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3263, 0.3447, 0.3290, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2513, 0.2541, 0.2382, 0.2564, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1990, 0.2053, 0.2053, 0.1933, 0.1971, 0.0000, 0.0000, 0.0000],
         [0.1652, 0.1714, 0.1651, 0.1654, 0.1702, 0.1628, 0.0000, 0.0000],
         [0.1405, 0.1400, 0.1408, 0.1332, 0.1508, 0.1433, 0.1513, 0.0000],
         [0.1248, 0.1204, 0.1213, 0.1217, 0.1194, 0.1269, 0.1358, 0.1297]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4836, 0.5164, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3261, 0.3475, 0.3263, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2441, 0.2564, 0.2337, 0.2658, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1932, 0.2039, 0.1842, 0.2121, 0.2066, 0.0000, 0.0000, 0.0000],
         [0.1619, 0.173